In [ ]:
library(GSVA)
library(clusterProfiler)
library(survival)
library(survminer)
library(Publish)
library(dplyr)
library(readxl)
library(ggsci)
library(compareGroups)

In [ ]:
setwd("/media/bio/Disk/Research Data/EBV/omicverse")

In [ ]:
# Prepare the bulk data and phenotype
GSE102349_Dataset <- read.csv(file="Dataset/GEO/GSE102349/02_normalized/GSE102349_normalized.csv",row.names=1,check.names=F)
# Same with the LUAD scRNA-seq data, the rows and columns are genes and cells, respectively. The dimensions of the LUAD bulk dataset are:
dim(GSE102349_Dataset)
GSE102349_Dataset <- GSE102349_Dataset %>% replace(., is.na(.), 0)
# It shows this data has 56716 genes and 471 samples in total. Users do not need to keep the common genes between the single-cell and bulk samples, which can be automatically achieved in Scissor. Besides, all of these samples have clinical outcomes:

# The clinical survival phenotype requires a two-column matrix with columns named 'time' and 'status'. The first column contains the sample IDs (observations), which should in the same order as the columns in the bulk expression matrix. The second column is a binary variable, with '1' indicating event (e.g.recurrence of cancer or death), and '0' indicating right-censored.
GSE102349_Clinical = read_excel("Dataset/GEO/GSE102349/clinical.xls",sheet = 1) 

In [ ]:
#GEO ssGSEA
Myeloid.Markers <- read.csv("Results/07.Myeloid/cosg_marker_gene_leiden_0_2.csv")
T.Markers <- read.csv("Results/11.TCell/cosg_marker_gene_leiden_1_0.csv")

In [ ]:
 Stem.Sig2 <- c(
  "DNMT3B", "PFAS", "XRCC5", "HAUS6", "TET1", "IGF2BP1", "PLAA", "TEX10", "MSH6", "DLGAP5",
  "SKIV2L2", "SOHLH2", "RRAS2", "PAICS", "CPSF3", "LIN28B", "IPO5", "BMPR1A", "ZNF788", "ASCC3",
  "FANCB", "HMGA2", "TRIM24", "ORC1", "HDAC2", "HESX1", "INHBE", "MIS18A", "DCUN1D5", "MRPL3", "CENPH",
  "MYCN", "HAUS1", "GDF3", "TBCE", "RIOK2", "BCKDHB", "RAD1", "NREP", "ADH5", "PLRG1",
  "ROR1", "RAB3B", "DIAPH3", "GNL2", "FGF2", "NMNAT2", "KIF20A", "CENPI", "DDX1", "XXYLT1",
  "GPR176", "BBS9", "C14orf166", "BOD1", "CDC123", "SNRPD3", "FAM118B", "DPH3", "EIF2B3", "RPF2",
  "APLP1", "DACT1", "PDHB", "C14orf119", "DTD1", "SAMM50", "CCL26", "MED20", "UTP6", "RARS2",
  "ARMCX2", "RARS", "MTHFD2", "DHX15", "HTR7", "MTHFD1L", "ARMC9", "XPOT", "IARS", "HDX",
  "ACTRT3", "ERCC2", "TBC1D16", "GARS", "KIF7", "UBE2K", "SLC25A3", "ICMT", "UGGT2", "ATP11C",
  "SLC24A1", "EIF2AK4", "GPX8", "ALX1", "OSTC", "TRPC4", "HAS2", "FZD2", "TRNT1", "MMADHC",
  "SNX8", "CDH6", "HAT1", "SEC11A", "DIMT1", "TM2D2", "FST", "GBE1"
)

In [ ]:
 Exhaustion <- c('PDCD1', 'LAYN', 'HAVCR2', 'LAG3', 'CD244', 'CTLA4', 'LILRB1', 'TIGIT', 'TOX', 'VSIR', 'BTLA', 'ENTPD1', 'CD160', 'LAIR1')

In [ ]:
GEP1 <- c("UBE2S", "NME1", "TUBB2B", "U2AF1", "ATP5G1", "CDKN2A", "MRPL12", "FABP6", "PTTG1", "TNFAIP6", 
"GADD45A", "HLA-DQA2", "SF3B14", "NDUFB8.1", "NHP2L1", "ATP5O", "MRPL23", "CNN3", "ATP5B", "CKS1B", 
"CCDC28B", "RGS16", "RPS4Y1", "ATPIF1", "ATP5G3", "HIST1H4C", "TK1", "PSMA2.1", "TAGLN2", "H2AFZ", 
"TUBA1C", "SYNGR2", "BIRC5", "LDHB", "DYNLT1", "EIF4A1", "ENO1", "ATP5I", "KPNA2", "RPL22L1", 
"TMC8", "C19orf10", "TXN", "KRT18", "REL", "KIAA0101", "UBE2C", "RPL27", "PRRX1", "USMG5"
)

In [ ]:
GEPs_Signature <- read.csv("Results/06.Epithelial/cNMF_Results/cNMF_topgenes_df.csv")
GEP1 <- GEPs_Signature$X1[1:100]
GEP1

In [ ]:
Meta_signature <-   list(Stem.Sig2 = Stem.Sig2,
                         GEP1 = GEP1,
                         SPP1_Macro = Myeloid.Markers$X12[1:50],
                         IL1B_Macro = Myeloid.Markers$X4[1:50],
                         C1QC_Macro = Myeloid.Markers$X2[1:50],
                         Tex = T.Markers$X26[1:50],
                         Tpex = T.Markers$X1[1:50],
                         TNFRSF9_N_Treg = T.Markers$X19[1:50],
                         TNFRSF9_P_Treg = T.Markers$X21[1:50],
                         Exhaustion = Exhaustion,
                         #M2_signature = Myeloid.Immunity.geneset$`M2 Macrophage Polarization`,
                         MYC = "MYC",
                         PDCD1 = "PDCD1",
                         LAG3 = "LAG3",
                         TOX = "TOX",
                         HAVCR2 = "HAVCR2",
                         CXCL13 = "CXCL13",
                         MIF = "MIF",
                         CD44 = "CD44",
                         C1QC = "C1QC",
                         IL1B = "IL1B",
                         SPP1 = "SPP1")

In [ ]:
y=as.matrix(GSE102349_Dataset)
geneSets=Meta_signature

In [ ]:
## build GSEA parameter object
ssgseaParam <- ssgseaParam(y, geneSets)
## estimate GSEA enrichment scores for the three sets
gsea_es <- gsva(ssgseaParam)
GSE102349_Scores <- as.data.frame(t(gsea_es))
GSE102349_Scores$SampleID <- rownames(GSE102349_Scores)

In [ ]:
ggplot(GSE102349_Scores, aes_string(x = "PDCD1", y = "SPP1_Macro")) +
  geom_point(size = 2,color = c('#EC0101'), alpha = 0.5) +
  theme_bw() + 
  theme(axis.title = element_text(size = 16),
        axis.text = element_text(size = 14),
        axis.ticks.length = unit(0.25,'cm'),
        axis.ticks = element_line(size = 1),
        panel.border = element_rect(size = 1.5),
        panel.grid = element_blank()
  ) + xlab("PDCD1") +
  geom_smooth(method = 'lm',se = T,color = '#F9B208',size = 1.5,fill = '#FEA82F') +
  stat_cor(method = "pearson",digits = 3,size=6)

ggsave("Results/09.GEO_validation/Scatter_PDCD1_SPP1.pdf", width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)


ggplot(GSE102349_Scores, aes_string(x = "LAG3", y = "SPP1_Macro")) +
  geom_point(size = 2,color = c('#EC0101'), alpha = 0.5) +
  theme_bw() + 
  theme(axis.title = element_text(size = 16),
        axis.text = element_text(size = 14),
        axis.ticks.length = unit(0.25,'cm'),
        axis.ticks = element_line(size = 1),
        panel.border = element_rect(size = 1.5),
        panel.grid = element_blank()
  ) + xlab("LAG3") +
  geom_smooth(method = 'lm',se = T,color = '#F9B208',size = 1.5,fill = '#FEA82F') +
  stat_cor(method = "pearson",digits = 3,size=6)

ggsave("Results/09.GEO_validation/Scatter_LAG3_SPP1.pdf", width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)


ggplot(GSE102349_Scores, aes_string(x = "HAVCR2", y = "SPP1_Macro")) +
  geom_point(size = 2,color = c('#EC0101'), alpha = 0.5) +
  theme_bw() + 
  theme(axis.title = element_text(size = 16),
        axis.text = element_text(size = 14),
        axis.ticks.length = unit(0.25,'cm'),
        axis.ticks = element_line(size = 1),
        panel.border = element_rect(size = 1.5),
        panel.grid = element_blank()
  ) + xlab("HAVCR2") +
  geom_smooth(method = 'lm',se = T,color = '#F9B208',size = 1.5,fill = '#FEA82F') +
  stat_cor(method = "pearson",digits = 3,size=6)

ggsave("Results/09.GEO_validation/Scatter_HAVCR2_SPP1.pdf", width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)



ggplot(GSE102349_Scores, aes_string(x = "Exhaustion", y = "SPP1_Macro")) +
  geom_point(size = 2,color = c('#EC0101'), alpha = 0.5) +
  theme_bw() + 
  theme(axis.title = element_text(size = 16),
        axis.text = element_text(size = 14),
        axis.ticks.length = unit(0.25,'cm'),
        axis.ticks = element_line(size = 1),
        panel.border = element_rect(size = 1.5),
        panel.grid = element_blank()
  ) + xlab("Exhaustion") +
  geom_smooth(method = 'lm',se = T,color = '#F9B208',size = 1.5,fill = '#FEA82F') +
  stat_cor(method = "pearson",digits = 3,size=6)

ggsave("Results/09.GEO_validation/Scatter_Exhaustion_SPP1.pdf", width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)



ggplot(GSE102349_Scores, aes_string(x = "Tex", y = "SPP1_Macro")) +
  geom_point(size = 2,color = c('#EC0101'), alpha = 0.5) +
  theme_bw() + 
  theme(axis.title = element_text(size = 16),
        axis.text = element_text(size = 14),
        axis.ticks.length = unit(0.25,'cm'),
        axis.ticks = element_line(size = 1),
        panel.border = element_rect(size = 1.5),
        panel.grid = element_blank()
  ) + xlab("Tex") +
  geom_smooth(method = 'lm',se = T,color = '#F9B208',size = 1.5,fill = '#FEA82F') +
  stat_cor(method = "pearson",digits = 3,size=6)

ggsave("Results/09.GEO_validation/Scatter_Tex_SPP1.pdf", width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)



ggplot(GSE102349_Scores, aes_string(x = "Tpex", y = "SPP1_Macro")) +
  geom_point(size = 2,color = c('#EC0101'), alpha = 0.5) +
  theme_bw() + 
  theme(axis.title = element_text(size = 16),
        axis.text = element_text(size = 14),
        axis.ticks.length = unit(0.25,'cm'),
        axis.ticks = element_line(size = 1),
        panel.border = element_rect(size = 1.5),
        panel.grid = element_blank()
  ) + xlab("Tpex") +
  geom_smooth(method = 'lm',se = T,color = '#F9B208',size = 1.5,fill = '#FEA82F') +
  stat_cor(method = "pearson",digits = 3,size=6)

ggsave("Results/09.GEO_validation/Scatter_Tpex_SPP1.pdf", width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)



ggplot(GSE102349_Scores, aes_string(x = "TNFRSF9_N_Treg", y = "SPP1_Macro")) +
  geom_point(size = 2,color = c('#EC0101'), alpha = 0.5) +
  theme_bw() + 
  theme(axis.title = element_text(size = 16),
        axis.text = element_text(size = 14),
        axis.ticks.length = unit(0.25,'cm'),
        axis.ticks = element_line(size = 1),
        panel.border = element_rect(size = 1.5),
        panel.grid = element_blank()
  ) + xlab("TNFRSF9_N_Treg") +
  geom_smooth(method = 'lm',se = T,color = '#F9B208',size = 1.5,fill = '#FEA82F') +
  stat_cor(method = "pearson",digits = 3,size=6)

ggsave("Results/09.GEO_validation/Scatter_TNFRSF9_N_Treg_SPP1.pdf", width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)



ggplot(GSE102349_Scores, aes_string(x = "TNFRSF9_P_Treg", y = "SPP1_Macro")) +
  geom_point(size = 2,color = c('#EC0101'), alpha = 0.5) +
  theme_bw() + 
  theme(axis.title = element_text(size = 16),
        axis.text = element_text(size = 14),
        axis.ticks.length = unit(0.25,'cm'),
        axis.ticks = element_line(size = 1),
        panel.border = element_rect(size = 1.5),
        panel.grid = element_blank()
  ) + xlab("TNFRSF9_P_Treg") +
  geom_smooth(method = 'lm',se = T,color = '#F9B208',size = 1.5,fill = '#FEA82F') +
  stat_cor(method = "pearson",digits = 3,size=6)

ggsave("Results/09.GEO_validation/Scatter_TNFRSF9_P_Treg_SPP1.pdf", width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:
ggplot(GSE102349_Scores, aes_string(x = "Stem.Sig2", y = "SPP1")) +
  geom_point(size = 2,color = c('#EC0101'), alpha = 0.5) +
  theme_bw() + 
  theme(axis.title = element_text(size = 16),
        axis.text = element_text(size = 14),
        axis.ticks.length = unit(0.25,'cm'),
        axis.ticks = element_line(size = 1),
        panel.border = element_rect(size = 1.5),
        panel.grid = element_blank()
  ) + xlab("Stemness Index") +
  geom_smooth(method = 'lm',se = T,color = '#F9B208',size = 1.5,fill = '#FEA82F') +
  stat_cor(method = "pearson",digits = 3,size=6)

ggsave("Results/09.GEO_validation/Scatter_Stem.Sig2_SPP1.pdf", width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:
ggplot(GSE102349_Scores, aes_string(x = "MIF", y = "SPP1")) +
  geom_point(size = 2,color = c('#EC0101'), alpha = 0.5) +
  theme_bw() + 
  theme(axis.title = element_text(size = 16),
        axis.text = element_text(size = 14),
        axis.ticks.length = unit(0.25,'cm'),
        axis.ticks = element_line(size = 1),
        panel.border = element_rect(size = 1.5),
        panel.grid = element_blank()
  ) +
  geom_smooth(method = 'lm',se = T,color = '#F9B208',size = 1.5,fill = '#FEA82F') +
  stat_cor(method = "pearson",digits = 3,size=6)

ggsave("Results/09.GEO_validation/Scatter_MIF_SPP1.pdf", width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:
GSE102349_Sig_Clinical <-  merge(x = GSE102349_Scores, y = GSE102349_Clinical, by = "SampleID", all = TRUE)

In [ ]:
GSE102349_Sig_Survival <- subset(GSE102349_Sig_Clinical,Time_months!="N/A")
# bulk_survival <- bulk_clinical
GSE102349_Sig_Survival$Time_months <- as.numeric(GSE102349_Sig_Survival$Time_months)
GSE102349_Sig_Survival$PFS_status <- as.numeric(GSE102349_Sig_Survival$PFS_status)

In [ ]:
Stem.Sig2.cut <- surv_cutpoint(GSE102349_Sig_Survival, time= 'Time_months', event = 'PFS_status', variables = 'Stem.Sig2') 
Stem.Sig2.cut 
GSE102349_Sig_Survival <- GSE102349_Sig_Survival %>% mutate(Stem.Sig2_2 = ifelse(Stem.Sig2 >= Stem.Sig2.cut$cutpoint[[1]], "High", "Low"))   
GSE102349_Sig_Survival$Stem.Sig2_2 <- factor(GSE102349_Sig_Survival$Stem.Sig2_2, levels = c("Low","High"))
res.cox1 <- coxph(Surv(Time_months,PFS_status)~Stem.Sig2_2, data = GSE102349_Sig_Survival)
publish(res.cox1)
fit_Stem.Sig2 <- survfit(Surv(Time_months,PFS_status) ~ Stem.Sig2_2, data = GSE102349_Sig_Survival)

ggsurv_Stem.Sig2_2 <- ggsurvplot(
  fit_Stem.Sig2, 
  palette = "jama",
  pval = T,
  pval.size = 5, 
  digits = 3,
  pval.method = T,
  pval.method.coord = c(3, 0.10),
  pval.coord = c(18, 0.10),
  conf.int = F,  
  censor = F,
  censor.shape = "|", 
  censor.size = 3, 
  test.for.trend = F,
  xlim = c(0, 50),
  ylim = c(0, 1.05),
  break.y.by = 0.2,
  surv.scale = "percent",
  lty = 1:1,
  xlab = "Time in months", 
  ylab = "Progression-Free Survival (%)",
  break.time.by = 12, 
  axes.offset = FALSE,
  risk.table = F, 
  risk.table.y.text.col = F,
  risk.table.y.text = F,
  risk.table.font.tickslab.x = FALSE,
  tables.y.text = F,
  risk.table.pos = "out", 
  tables.height = 0.25, 
  font.risk.table.x = c(12, "plain", "black"),
  font.x = c(16, "plain", "black"),
  font.y = c(16, "plain", "black"),
  font.tickslab = c(14, "plain", "black"),
  legend = "top",            
  legend.title = "Stemness Index",        
  legend.labs = c("Low", "High"),
  font.legend = c(16, "plain", "black"),
  font.legend.title = c(20, "plain", "black"),
  font.legend.labs = c(16, "plain", "black"),
  linetype = "solid",
  tables.theme = theme_cleantable()
)

ggsurv_Stem.Sig2_2$plot <- ggsurv_Stem.Sig2_2$plot +
  scale_y_continuous(
    expand = c(0, 0), 
    labels = function(x) x * 100 
  ) +
  theme(
    plot.margin = margin(20, 20, 20, 20), 
    legend.position = c(0.05, 0.12), 
    legend.justification = c(0, 0), 
    legend.background = element_blank(),
    legend.key = element_blank(),
    legend.key.height = unit(1, "cm"), 
    legend.text = element_text(size = 16) 
  ) +
  guides(
    color = guide_legend(ncol = 2, byrow = TRUE) 
  )

ggsurv_Stem.Sig2_2$table <- ggsurv_Stem.Sig2_2$table +
  theme(
    plot.margin = margin(0, 20, 20, 20)
  )

ggsurv_Stem.Sig2_2
ggsave("Results/09.GEO_validation/Survival_Curves_Stem.Sig2.pdf", ggsurv_Stem.Sig2_2$plot, width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:
GEP1.cut <- surv_cutpoint(GSE102349_Sig_Survival, time = 'Time_months', event = 'PFS_status', variables = 'GEP1') 
GEP1.cut  
GSE102349_Sig_Survival <- GSE102349_Sig_Survival %>%
  mutate(GEP1_2 = ifelse(GEP1 >= GEP1.cut$cutpoint[[1]], "High", "Low"))   
GSE102349_Sig_Survival$GEP1_2 <- factor(GSE102349_Sig_Survival$GEP1_2, levels = c("Low", "High"))

res.cox1 <- coxph(Surv(Time_months, PFS_status) ~ GEP1_2, data = GSE102349_Sig_Survival)
publish(res.cox1)

fit_GEP1 <- survfit(Surv(Time_months, PFS_status) ~ GEP1_2, data = GSE102349_Sig_Survival)

ggsurv_GEP1_2 <- ggsurvplot(
  fit_GEP1, 
  palette = "jama",
  pval = TRUE,
  pval.size = 5, 
  digits = 3,
  pval.method = TRUE,
  pval.method.coord = c(3, 0.10),
  pval.coord = c(18, 0.10),
  conf.int = FALSE,  
  censor = FALSE,
  censor.shape = "|", 
  censor.size = 3, 
  test.for.trend = FALSE,
  xlim = c(0, 50),
  ylim = c(0, 1.05),
  break.y.by = 0.2,
  surv.scale = "percent",
  lty = 1,
  xlab = "Time in months", 
  ylab = "Progression-Free Survival (%)",
  break.time.by = 12, 
  axes.offset = FALSE,
  risk.table = FALSE, 
  risk.table.y.text.col = FALSE,
  risk.table.y.text = FALSE,
  risk.table.font.tickslab.x = FALSE,
  tables.y.text = FALSE,
  risk.table.pos = "out", 
  tables.height = 0.25,
  font.risk.table.x = c(12, "plain", "black"),
  font.x = c(16, "plain", "black"),
  font.y = c(16, "plain", "black"),
  font.tickslab = c(14, "plain", "black"),
  legend = "top",            
  legend.title = "GEP1 Score",        
  legend.labs = c("Low", "High"),
  font.legend = c(16, "plain", "black"),
  font.legend.title = c(20, "plain", "black"),
  font.legend.labs = c(16, "plain", "black"),
  linetype = "solid",
  tables.theme = theme_cleantable()
)

ggsurv_GEP1_2$plot <- ggsurv_GEP1_2$plot +
  scale_y_continuous(
    expand = c(0, 0),
    labels = function(x) x * 100
  ) +
  theme(
    plot.margin = margin(20, 20, 20, 20),
    legend.position = c(0.05, 0.12),
    legend.justification = c(0, 0),
    legend.background = element_blank(),
    legend.key = element_blank(),
    legend.key.height = unit(1, "cm"),
    legend.text = element_text(size = 16)
  ) +
  guides(
    color = guide_legend(ncol = 2, byrow = TRUE)
  )

ggsurv_GEP1_2$table <- ggsurv_GEP1_2$table +
  theme(
    plot.margin = margin(0, 20, 20, 20)
  )

ggsurv_GEP1_2
ggsave("Results/09.GEO_validation/Survival_Curves_GEP1.pdf", ggsurv_GEP1_2$plot,
       width = 5, height = 5, units = "in", dpi = 300, limitsize = TRUE)

In [ ]:
ggplot(GSE102349_Scores, aes_string(x = "GEP1", y = "Stem.Sig2")) +
  geom_point(size = 2,color = c('#EC0101'), alpha = 0.5) +
  theme_bw() + 
  theme(axis.title = element_text(size = 16),
        axis.text = element_text(size = 14),
        axis.ticks.length = unit(0.25,'cm'),
        axis.ticks = element_line(size = 1),
        panel.border = element_rect(size = 1.5),
        panel.grid = element_blank()
  ) + xlab("GEP1 Score") + ylab("Stemness Index") +
  geom_smooth(method = 'lm',se = T,color = '#F9B208',size = 1.5,fill = '#FEA82F') +
  stat_cor(method = "pearson",digits = 3,size=6)

ggsave("Results/09.GEO_validation/Scatter_GEP1_Stem.Sig2.pdf", width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:
MIF.cut <- surv_cutpoint(GSE102349_Sig_Survival, time= 'Time_months', event = 'PFS_status', variables = 'MIF') 
MIF.cut 
GSE102349_Sig_Survival <- GSE102349_Sig_Survival %>% mutate(MIF_2 = ifelse(MIF >= MIF.cut$cutpoint[[1]], "High", "Low"))   
GSE102349_Sig_Survival$MIF_2 <- factor(GSE102349_Sig_Survival$MIF_2, levels = c("Low","High"))
res.cox1 <- coxph(Surv(Time_months,PFS_status)~MIF_2, data = GSE102349_Sig_Survival)
publish(res.cox1)
fit_MIF_2 <- survfit(Surv(Time_months,PFS_status) ~ MIF_2, data = GSE102349_Sig_Survival)
ggsurv_MIF_2 <- ggsurvplot(
  fit_MIF_2, 
  palette = "jama",
  pval = T,
  pval.size = 5, 
  digits = 3,
  pval.method = T,
  pval.method.coord = c(3, 0.1),
  pval.coord = c(18, 0.1),
  conf.int = F,  
  censor = F,
  censor.shape = "|", 
  censor.size = 3, 
  test.for.trend = F,
  xlim = c(0, 50),
  ylim = c(0, 1.05),
  break.y.by = 0.2,
  surv.scale = "percent",
  lty = 1:1,
  xlab = "Time in months", 
  ylab = "Progression-Free Survival (%)",
  break.time.by = 12, 
  axes.offset = FALSE,
  risk.table = F, 
  risk.table.y.text.col = F,
  risk.table.y.text = F,
  risk.table.font.tickslab.x = FALSE,
  tables.y.text = F,
  risk.table.pos = "out", 
  tables.height = 0.25,
  font.risk.table.x = c(12, "plain", "black"),
  font.x = c(16, "plain", "black"),
  font.y = c(16, "plain", "black"),
  font.tickslab = c(14, "plain", "black"),
  legend = "top",            
  legend.title = "MIF Signature",        
  legend.labs = c("Low", "High"),
  font.legend = c(16, "plain", "black"),
  font.legend.title = c(20, "plain", "black"),
  font.legend.labs = c(16, "plain", "black"),
  linetype = "solid",
  tables.theme = theme_cleantable()
)

ggsurv_MIF_2$plot <- ggsurv_MIF_2$plot +
  scale_y_continuous(
    expand = c(0, 0), 
    labels = function(x) x * 100 
  ) +
  theme(
    plot.margin = margin(20, 20, 20, 20),
    legend.position = c(0.05, 0.12),
    legend.justification = c(0, 0), 
    legend.background = element_blank(),
    legend.key = element_blank(),
    legend.key.height = unit(1, "cm"), 
    legend.text = element_text(size = 16) 
  ) +
  guides(
    color = guide_legend(ncol = 2, byrow = TRUE)
  )

ggsurv_MIF_2$table <- ggsurv_MIF_2$table +
  theme(
    plot.margin = margin(0, 20, 20, 20) 
  )
ggsurv_MIF_2
ggsave("Results/09.GEO_validation/Survival_Curves_MIF_2.pdf", ggsurv_MIF_2$plot, width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:
SPP1.cut <- surv_cutpoint(GSE102349_Sig_Survival, time= 'Time_months', event = 'PFS_status', variables = 'SPP1') 
SPP1.cut
GSE102349_Sig_Survival <- GSE102349_Sig_Survival %>% mutate(SPP1_2 = ifelse(SPP1 >= SPP1.cut$cutpoint[[1]], "High", "Low"))   
GSE102349_Sig_Survival$SPP1_2 <- factor(GSE102349_Sig_Survival$SPP1_2, levels = c("Low","High"))
res.cox1 <- coxph(Surv(Time_months,PFS_status)~SPP1_2, data = GSE102349_Sig_Survival)
publish(res.cox1)
fit_SPP1_2 <- survfit(Surv(Time_months,PFS_status) ~ SPP1_2, data = GSE102349_Sig_Survival)
ggsurv_SPP1_2 <- ggsurvplot(
  fit_SPP1_2, 
  palette = "jama",
  pval = T,
  pval.size = 5, 
  digits = 3,
  pval.method = T,
  pval.method.coord = c(3, 0.1),
  pval.coord = c(18, 0.1),
  conf.int = F,  
  censor = F,
  censor.shape = "|", 
  censor.size = 3, 
  test.for.trend = F,
  xlim = c(0, 50),
  ylim = c(0, 1.05),
  break.y.by = 0.2,
  surv.scale = "percent",
  lty = 1:1,
  xlab = "Time in months", 
  ylab = "Progression-Free Survival (%)",
  break.time.by = 12, 
  axes.offset = FALSE,
  risk.table = F, 
  risk.table.y.text.col = F,
  risk.table.y.text = F,
  risk.table.font.tickslab.x = FALSE,
  tables.y.text = F,
  risk.table.pos = "out", 
  tables.height = 0.25, 
  font.risk.table.x = c(12, "plain", "black"),
  font.x = c(16, "plain", "black"),
  font.y = c(16, "plain", "black"),
  font.tickslab = c(14, "plain", "black"),
  legend = "top",            
  legend.title = "SPP1 Signature",        
  legend.labs = c("Low", "High"),
  font.legend = c(16, "plain", "black"),
  font.legend.title = c(20, "plain", "black"),
  font.legend.labs = c(16, "plain", "black"),
  linetype = "solid",
  tables.theme = theme_cleantable()
)

ggsurv_SPP1_2$plot <- ggsurv_SPP1_2$plot +
  scale_y_continuous(
    expand = c(0, 0),
    labels = function(x) x * 100
  ) +
  theme(
    plot.margin = margin(20, 20, 20, 20),
    legend.position = c(0.05, 0.12),
    legend.justification = c(0, 0), 
    #legend.background = element_blank(),
    #legend.key = element_blank(),
    legend.key.height = unit(1, "cm"), 
    legend.text = element_text(size = 16) 
  ) +
  guides(
    color = guide_legend(ncol = 2, byrow = TRUE)
  )

ggsurv_SPP1_2$table <- ggsurv_SPP1_2$table +
  theme(
    plot.margin = margin(0, 20, 20, 20)
  )
ggsurv_SPP1_2
ggsave("Results/09.GEO_validation/Survival_Curves_SPP1_2.pdf", ggsurv_SPP1_2$plot, width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:
SPP1_Macro.cut <- surv_cutpoint(GSE102349_Sig_Survival, time= 'Time_months', event = 'PFS_status', variables = 'SPP1_Macro') 
SPP1_Macro.cut
GSE102349_Sig_Survival <- GSE102349_Sig_Survival %>% mutate(SPP1_Macro_2 = ifelse(SPP1_Macro >= SPP1_Macro.cut$cutpoint[[1]], "High", "Low"))   
GSE102349_Sig_Survival$SPP1_Macro_2 <- factor(GSE102349_Sig_Survival$SPP1_2, levels = c("Low","High"))
res.cox1 <- coxph(Surv(Time_months,PFS_status)~SPP1_Macro_2, data = GSE102349_Sig_Survival)
publish(res.cox1)
fit_SPP1_Macro_2 <- survfit(Surv(Time_months,PFS_status) ~ SPP1_Macro_2, data = GSE102349_Sig_Survival)
ggsurv_SPP1_Macro_2 <- ggsurvplot(
  fit_SPP1_Macro_2, 
  palette = "jama",
  pval = T,
  pval.size = 5, 
  digits = 3,
  pval.method = T,
  pval.method.coord = c(3, 0.1),
  pval.coord = c(18, 0.1),
  conf.int = F,  
  censor = F,
  censor.shape = "|", 
  censor.size = 3, 
  test.for.trend = F,
  xlim = c(0, 50),
  ylim = c(0, 1.05),
  break.y.by = 0.2,
  surv.scale = "percent",
  lty = 1:1,
  xlab = "Time in months", 
  ylab = "Progression-Free Survival (%)",
  break.time.by = 12, 
  axes.offset = FALSE,
  risk.table = F, 
  risk.table.y.text.col = F,
  risk.table.y.text = F,
  risk.table.font.tickslab.x = FALSE,
  tables.y.text = F,
  risk.table.pos = "out", 
  tables.height = 0.25,
  font.risk.table.x = c(12, "plain", "black"),
  font.x = c(16, "plain", "black"),
  font.y = c(16, "plain", "black"),
  font.tickslab = c(14, "plain", "black"),
  legend = "top",            
  legend.title = "SPP1+ Macrophage Signature",        
  legend.labs = c("Low", "High"),
  font.legend = c(16, "plain", "black"),
  font.legend.title = c(20, "plain", "black"),
  font.legend.labs = c(16, "plain", "black"),
  linetype = "solid",
  tables.theme = theme_cleantable()
)

ggsurv_SPP1_Macro_2$plot <- ggsurv_SPP1_Macro_2$plot +
  scale_y_continuous(
    expand = c(0, 0), 
    labels = function(x) x * 100 
  ) +
  theme(
    plot.margin = margin(20, 20, 20, 20), 
    legend.position = c(0.05, 0.12),
    legend.justification = c(0, 0), 
    legend.background = element_blank(),
    legend.key = element_blank(),
    legend.key.height = unit(1, "cm"), 
    legend.text = element_text(size = 16) 
  ) +
  guides(
    color = guide_legend(ncol = 2, byrow = TRUE)
  )

ggsurv_SPP1_Macro_2$table <- ggsurv_SPP1_Macro_2$table +
  theme(
    plot.margin = margin(0, 20, 20, 20)
  )
ggsurv_SPP1_Macro_2
ggsave("Results/09.GEO_validation/Survival_Curves_SPP1_Macro_2.pdf", ggsurv_SPP1_Macro_2$plot, width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:

C1QC.cut <- surv_cutpoint(GSE102349_Sig_Survival, time= 'Time_months', event = 'PFS_status', variables = 'C1QC') 
C1QC.cut
GSE102349_Sig_Survival <- GSE102349_Sig_Survival %>% mutate(C1QC_2 = ifelse(C1QC >= C1QC.cut$cutpoint[[1]], "High", "Low"))   
GSE102349_Sig_Survival$C1QC_2 <- factor(GSE102349_Sig_Survival$C1QC_2, levels = c("Low","High"))
res.cox1 <- coxph(Surv(Time_months,PFS_status)~C1QC_2, data = GSE102349_Sig_Survival)
publish(res.cox1)
fit_C1QC <- survfit(Surv(Time_months,PFS_status) ~ C1QC_2, data = GSE102349_Sig_Survival)
ggsurv_C1QC <- ggsurvplot(
  fit_C1QC, 
  palette = "jama",
  pval = T,
  pval.size = 5, 
  digits = 3,
  pval.method = T,
  pval.method.coord = c(3, 0.1),
  pval.coord = c(18, 0.1),
  conf.int = F,  
  censor = F,
  censor.shape = "|", 
  censor.size = 3, 
  test.for.trend = F,
  xlim = c(0, 50),
  ylim = c(0, 1.05),
  break.y.by = 0.2,
  surv.scale = "percent",
  lty = 1:1,
  xlab = "Time in months", 
  ylab = "Progression-Free Survival (%)",
  break.time.by = 12, 
  axes.offset = FALSE,
  risk.table = F, 
  risk.table.y.text.col = F,
  risk.table.y.text = F,
  risk.table.font.tickslab.x = FALSE,
  tables.y.text = F,
  risk.table.pos = "out", 
  tables.height = 0.25,
  font.risk.table.x = c(12, "plain", "black"),
  font.x = c(16, "plain", "black"),
  font.y = c(16, "plain", "black"),
  font.tickslab = c(14, "plain", "black"),
  legend = "top",            
  legend.title = "C1QC+ Macrophage Signature",        
  legend.labs = c("Low", "High"),
  font.legend = c(16, "plain", "black"),
  font.legend.title = c(20, "plain", "black"),
  font.legend.labs = c(16, "plain", "black"),
  linetype = "solid",
  tables.theme = theme_cleantable()
)

ggsurv_C1QC$plot <- ggsurv_C1QC$plot +
  scale_y_continuous(
    expand = c(0, 0), 
    labels = function(x) x * 100 
  ) +
  theme(
    plot.margin = margin(20, 20, 20, 20), 
    legend.position = c(0.05, 0.12), 
    legend.justification = c(0, 0), 
    legend.background = element_blank(),
    legend.key = element_blank(),
    legend.key.height = unit(1, "cm"), 
    legend.text = element_text(size = 16) 
  ) +
  guides(
    color = guide_legend(ncol = 2, byrow = TRUE) 
  )

ggsurv_C1QC$table <- ggsurv_C1QC$table +
  theme(
    plot.margin = margin(0, 20, 20, 20) 
  )

ggsurv_C1QC
ggsave("Results/09.GEO_validation/Survival_Curves_C1QC.pdf", ggsurv_C1QC$plot, width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)


In [ ]:
C1QC_Macro.cut <- surv_cutpoint(GSE102349_Sig_Survival, time= 'Time_months', event = 'PFS_status', variables = 'C1QC_Macro') 
C1QC_Macro.cut
GSE102349_Sig_Survival <- GSE102349_Sig_Survival %>% mutate(C1QC_Macro_2 = ifelse(C1QC_Macro >= C1QC_Macro.cut$cutpoint[[1]], "High", "Low"))   
GSE102349_Sig_Survival$C1QC_Macro_2 <- factor(GSE102349_Sig_Survival$C1QC_Macro_2, levels = c("Low","High"))
res.cox1 <- coxph(Surv(Time_months,PFS_status)~C1QC_Macro_2, data = GSE102349_Sig_Survival)
publish(res.cox1)
fit_C1QC_Macro_2 <- survfit(Surv(Time_months,PFS_status) ~ C1QC_Macro_2, data = GSE102349_Sig_Survival)
ggsurv_C1QC_Macro_2 <- ggsurvplot(
  fit_C1QC_Macro_2, 
  palette = "jama",
  pval = T,
  pval.size = 5, 
  digits = 3,
  pval.method = T,
  pval.method.coord = c(3, 0.1),
  pval.coord = c(18, 0.1),
  conf.int = F,  
  censor = F,
  censor.shape = "|", 
  censor.size = 3, 
  test.for.trend = F,
  xlim = c(0, 50),
  ylim = c(0, 1.05),
  break.y.by = 0.2,
  surv.scale = "percent",
  lty = 1:1,
  xlab = "Time in months", 
  ylab = "Progression-Free Survival (%)",
  break.time.by = 12, 
  axes.offset = FALSE,
  risk.table = F, 
  risk.table.y.text.col = F,
  risk.table.y.text = F,
  risk.table.font.tickslab.x = FALSE,
  tables.y.text = F,
  risk.table.pos = "out", 
  tables.height = 0.25,
  font.risk.table.x = c(12, "plain", "black"),
  font.x = c(16, "plain", "black"),
  font.y = c(16, "plain", "black"),
  font.tickslab = c(14, "plain", "black"),
  legend = "top",            
  legend.title = "C1QC+ Macrophage Signature",        
  legend.labs = c("Low", "High"),
  font.legend = c(16, "plain", "black"),
  font.legend.title = c(20, "plain", "black"),
  font.legend.labs = c(16, "plain", "black"),
  linetype = "solid",
  tables.theme = theme_cleantable()
)

ggsurv_C1QC_Macro_2$plot <- ggsurv_C1QC_Macro_2$plot +
  scale_y_continuous(
    expand = c(0, 0),
    labels = function(x) x * 100
  ) +
  theme(
    plot.margin = margin(20, 20, 20, 20), 
    legend.position = c(0.05, 0.12),
    legend.justification = c(0, 0), 
    legend.background = element_blank(),
    legend.key = element_blank(),
    legend.key.height = unit(1, "cm"), 
    legend.text = element_text(size = 16) 
  ) +
  guides(
    color = guide_legend(ncol = 2, byrow = TRUE)
  )

ggsurv_C1QC_Macro_2$table <- ggsurv_C1QC_Macro_2$table +
  theme(
    plot.margin = margin(0, 20, 20, 20) 
  )
ggsurv_C1QC_Macro_2
ggsave("Results/09.GEO_validation/Survival_Curves_C1QC_Macro_2.pdf", ggsurv_C1QC_Macro_2$plot, width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)


In [ ]:
IL1B.cut <- surv_cutpoint(GSE102349_Sig_Survival, time= 'Time_months', event = 'PFS_status', variables = 'IL1B') 
IL1B.cut 
GSE102349_Sig_Survival <- GSE102349_Sig_Survival %>% mutate(IL1B_2 = ifelse(IL1B >= IL1B.cut$cutpoint[[1]], "High", "Low"))   
GSE102349_Sig_Survival$IL1B_2 <- factor(GSE102349_Sig_Survival$IL1B_2, levels = c("Low","High"))
res.cox1 <- coxph(Surv(Time_months,PFS_status)~IL1B_2, data = GSE102349_Sig_Survival)
publish(res.cox1)
fit_IL1B_2 <- survfit(Surv(Time_months,PFS_status) ~ IL1B_2, data = GSE102349_Sig_Survival)
ggsurv_IL1B_2 <- ggsurvplot(
  fit_IL1B_2, 
  palette = "jama",
  pval = T,
  pval.size = 5, 
  digits = 3,
  pval.method = T,
  pval.method.coord = c(3, 0.1),
  pval.coord = c(18, 0.1),
  conf.int = F,  
  censor = F,
  censor.shape = "|", 
  censor.size = 3, 
  test.for.trend = F,
  xlim = c(0, 50),
  ylim = c(0, 1.05),
  break.y.by = 0.2,
  surv.scale = "percent",
  lty = 1:1,
  xlab = "Time in months", 
  ylab = "Progression-Free Survival (%)",
  break.time.by = 12, 
  axes.offset = FALSE,
  risk.table = F, 
  risk.table.y.text.col = F,
  risk.table.y.text = F,
  risk.table.font.tickslab.x = FALSE,
  tables.y.text = F,
  risk.table.pos = "out", 
  tables.height = 0.25, 
  font.risk.table.x = c(12, "plain", "black"),
  font.x = c(16, "plain", "black"),
  font.y = c(16, "plain", "black"),
  font.tickslab = c(14, "plain", "black"),
  legend = "top",            
  legend.title = "IL1B Signature",        
  legend.labs = c("Low", "High"),
  font.legend = c(16, "plain", "black"),
  font.legend.title = c(20, "plain", "black"),
  font.legend.labs = c(16, "plain", "black"),
  linetype = "solid",
  tables.theme = theme_cleantable()
)

ggsurv_IL1B_2$plot <- ggsurv_IL1B_2$plot +
  scale_y_continuous(
    expand = c(0, 0), 
    labels = function(x) x * 100 
  ) +
  theme(
    plot.margin = margin(20, 20, 20, 20), 
    legend.position = c(0.05, 0.12), 
    legend.justification = c(0, 0), 
    legend.background = element_blank(),
    legend.key = element_blank(),
    legend.key.height = unit(1, "cm"), 
    legend.text = element_text(size = 16) 
  ) +
  guides(
    color = guide_legend(ncol = 2, byrow = TRUE)
  )

ggsurv_IL1B_2$table <- ggsurv_IL1B_2$table +
  theme(
    plot.margin = margin(0, 20, 20, 20)
  )
ggsurv_IL1B_2
ggsave("Results/09.GEO_validation/Survival_Curves_IL1B_2.pdf", ggsurv_IL1B_2$plot, width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:
IL1B_Macro.cut <- surv_cutpoint(GSE102349_Sig_Survival, time= 'Time_months', event = 'PFS_status', variables = 'IL1B_Macro') 
IL1B_Macro.cut 
GSE102349_Sig_Survival <- GSE102349_Sig_Survival %>% mutate(IL1B_Macro_2 = ifelse(IL1B_Macro >= IL1B_Macro.cut$cutpoint[[1]], "High", "Low"))   
GSE102349_Sig_Survival$IL1B_Macro_2 <- factor(GSE102349_Sig_Survival$IL1B_Macro_2, levels = c("Low","High"))
res.cox1 <- coxph(Surv(Time_months,PFS_status)~IL1B_Macro_2, data = GSE102349_Sig_Survival)
publish(res.cox1)
fit_IL1B_Macro_2 <- survfit(Surv(Time_months,PFS_status) ~ IL1B_Macro_2, data = GSE102349_Sig_Survival)
ggsurv_IL1B_Macro_2 <- ggsurvplot(
  fit_IL1B_Macro_2, 
  palette = "jama",
  pval = T,
  pval.size = 5, 
  digits = 3,
  pval.method = T,
  pval.method.coord = c(3, 0.1),
  pval.coord = c(18, 0.1),
  conf.int = F,  
  censor = F,
  censor.shape = "|", 
  censor.size = 3, 
  test.for.trend = F,
  xlim = c(0, 50),
  ylim = c(0, 1.05),
  break.y.by = 0.2,
  surv.scale = "percent",
  lty = 1:1,
  xlab = "Time in months", 
  ylab = "Progression-Free Survival (%)",
  break.time.by = 12, 
  axes.offset = FALSE,
  risk.table = F, 
  risk.table.y.text.col = F,
  risk.table.y.text = F,
  risk.table.font.tickslab.x = FALSE,
  tables.y.text = F,
  risk.table.pos = "out", 
  tables.height = 0.25, 
  font.risk.table.x = c(12, "plain", "black"),
  font.x = c(16, "plain", "black"),
  font.y = c(16, "plain", "black"),
  font.tickslab = c(14, "plain", "black"),
  legend = "top",            
  legend.title = "IL1B+ Macrophage Signature",        
  legend.labs = c("Low", "High"),
  font.legend = c(16, "plain", "black"),
  font.legend.title = c(20, "plain", "black"),
  font.legend.labs = c(16, "plain", "black"),
  linetype = "solid",
  tables.theme = theme_cleantable()
)


ggsurv_IL1B_Macro_2$plot <- ggsurv_IL1B_Macro_2$plot +
  scale_y_continuous(
    expand = c(0, 0),
    labels = function(x) x * 100
  ) +
  theme(
    plot.margin = margin(20, 20, 20, 20),
    legend.position = c(0.05, 0.12),
    legend.justification = c(0, 0), 
    legend.background = element_blank(),
    legend.key = element_blank(),
    legend.key.height = unit(1, "cm"), 
    legend.text = element_text(size = 16) 
  ) +
  guides(
    color = guide_legend(ncol = 2, byrow = TRUE)
  )

ggsurv_IL1B_Macro_2$table <- ggsurv_IL1B_Macro_2$table +
  theme(
    plot.margin = margin(0, 20, 20, 20)
  )
ggsurv_IL1B_Macro_2
ggsave("Results/09.GEO_validation/Survival_Curves_IL1B_Macro_2.pdf", ggsurv_IL1B_Macro_2$plot, width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

### Correlation analysis between Stem.Sig1/SPP1/SPP1_Macro and tumor stage

In [ ]:
GSE102349_Sig_Survival1 <- subset(GSE102349_Sig_Survival, GSE102349_Sig_Survival$Stage!="Unknown")

GSE102349_Sig_Survival1 <- GSE102349_Sig_Survival1 %>%
  mutate(Stage2 = case_when(
    Stage %in% c("I", "II") ~ "I-II",
    Stage == "III" ~ "III-IV",
    Stage == "IV" ~ "III-IV",
    TRUE ~ NA_character_ 
  ))

GSE102349_Sig_Survival1$Stage2 <- factor(GSE102349_Sig_Survival1$Stage2)


GSE102349_Sig_Survival1 <- GSE102349_Sig_Survival1 %>%
  mutate(Stage3 = case_when(
    Stage %in% c("I", "II") ~ "I-II",
    Stage == "III" ~ "III",
    Stage == "IV" ~ "IV",
    TRUE ~ NA_character_ 
  ))

GSE102349_Sig_Survival1$Stage3 <- factor(GSE102349_Sig_Survival1$Stage3)

In [ ]:
ggviolin_SPP1 <- ggviolin(GSE102349_Sig_Survival1,
                     x = "Stage3", 
                     y = "SPP1",
                     xlab = "Clincial Stage", 
                     ylab = "SPP1 Signature",
                     color = "Stage3", 
                     palette = "jama",
                     add = c("boxplot", "jitter"),
                     shape = "Stage3"
) + 
  stat_compare_means(comparisons = list(c('I-II','III'), c('I-II','IV')), 
                     method = "wilcox.test", label = "p.format", label.y = c(1.2,1.3)) +
  theme(legend.position = "none") 
ggviolin_SPP1
ggsave("Results/09.GEO_validation/ggviolin_SPP1_stage.pdf", ggviolin_SPP1, width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)


In [ ]:
ggviolin_Stem.Sig2 <- ggviolin(GSE102349_Sig_Survival1,
                     x = "Stage3", 
                     y = "Stem.Sig2",
                     xlab = "Clinical Stage",
                     ylab = "Stemness Index",
                     color = "Stage3", 
                     palette = "jama",
                     add = c("boxplot", "jitter"),
                     shape = "Stage3"
) + 
  stat_compare_means(comparisons = list(c('I-II','III'), c('I-II','IV')), 
                     method = "wilcox.test", label = "p.format", label.y = c(0.4,0.42)) +
  theme(legend.position = "none") 
ggviolin_Stem.Sig2
ggsave("Results/09.GEO_validation/ggviolin_Stem.Sig2_satge.pdf", ggviolin_Stem.Sig2, width = 5, height = 5, units = c("in"), dpi = 300, limitsize = TRUE)

In [ ]:
sessionInfo()